<a href="https://colab.research.google.com/github/HamzaaBl/Sentiment-Analysis-on-IMDB-Reviews-with-LSTM/blob/main/Sentiment_Analysis_on_IMDB_Reviews_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install Kaggle

**Importing dependencies**


In [5]:
import os
import json

from zipfile import ZipFile #To get the Data from Kaggle API
import pandas as pd #Load the CSV file into a panda data frame
from sklearn.model_selection import train_test_split #Split our Data into training data and testing Data
from tensorflow.keras.models import Sequential #Build the sequential layer of neural network
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer #Tokenize the Data: Map words in the data to certain integers or vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences #Ensure that all your input Data is of the same shape

**Data Collection - Kaggle API**

In [8]:
# Setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"]=kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"]=kaggle_dictionary["key"]

In [10]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 51% 13.0M/25.7M [00:00<00:00, 133MB/s]
100% 25.7M/25.7M [00:00<00:00, 175MB/s]


In [11]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [12]:
#unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

In [13]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


**Loading the Dataset**

In [15]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [16]:
data.shape

(50000, 2)

In [17]:
data.head() #Get an idea of how the Data goes

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [19]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [20]:
data.replace({"sentiment":{"positive": 1, "negative": 0}}, inplace=True)

<ipython-input-20-b85c8d25736e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive": 1, "negative": 0}}, inplace=True)


In [21]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [22]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [23]:
#split data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [24]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**


In [30]:
# Tokenize text data
tokenizer = Tokenizer(num_words=5000) #Convert wors or phrases into particular numbers
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [31]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [32]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [33]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [34]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


**LSTM - Long Short-Term Memory**

In [39]:
#build the model

model= Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200)) #First layer in your network/ represnting Data in Vector form
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid")) #All the neurons in the previous layer are connected in this layer

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [41]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [43]:
# compile the model
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

**Train the Model**

In [44]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 149s 281ms/step - accuracy: 0.7190 - loss: 0.5323 - val_accuracy: 0.8426 - val_loss: 0.3830
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 279ms/step - accuracy: 0.8390 - loss: 0.3740 - val_accuracy: 0.8495 - val_loss: 0.3574
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 143s 282ms/step - accuracy: 0.8766 - loss: 0.3041 - val_accuracy: 0.8676 - val_loss: 0.3411
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 138s 274ms/step - accuracy: 0.8967 - loss: 0.2662 - val_accuracy: 0.8629 - val_loss: 0.3270
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 141s 272ms/step - accuracy: 0.9005 - loss: 0.2511 - val_accuracy: 0.8649 - val_loss: 0.3241


**Model Evaluation**

In [45]:
loss, accuracy= model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 86ms/step - accuracy: 0.8678 - loss: 0.3154
Test Loss: 0.3150215744972229
Test Accuracy: 0.8711000084877014


**Building a Predictive Sytem**

In [47]:
def predict_sentiment(review):
  #tokenize and pad the review text
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)

  #make a prediction
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction > 0.5 else "negative"
  return sentiment



In [49]:
# example usage
new_review ="This movie was fantastic. I loved it"
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
The sentiment of the review is: positive


In [58]:
# example usage
new_review ="I love the girl in the movie. But the movie is horrible"
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
The sentiment of the review is: negative
